In [1]:
import gpxpy
import gpxpy.gpx
import pandas as pd
import numpy as np
from typing import Tuple, List
from haversine import haversine, Unit
import folium

def load_gpx(file_path: str) -> gpxpy.gpx.GPX:
    with open(file_path, 'r', encoding='utf-8') as gpx_file:
        return gpxpy.parse(gpx_file)

def find_closest_point(gpx: gpxpy.gpx.GPX, ref_coord: Tuple[float, float]) -> int:
    min_dist = float('inf')
    closest_idx = -1
    points = [(p.latitude, p.longitude) for track in gpx.tracks for seg in track.segments for p in seg.points]
    for idx, point in enumerate(points):
        dist = haversine(ref_coord, point, unit=Unit.METERS)
        if dist < min_dist:
            min_dist = dist
            closest_idx = idx
    return closest_idx

def slice_track(gpx: gpxpy.gpx.GPX, start_idx: int, length: int = 50) -> List[gpxpy.gpx.GPXTrackPoint]:
    points = [p for track in gpx.tracks for seg in track.segments for p in seg.points]
    return points[start_idx: start_idx + length]

def compute_features(points: List[gpxpy.gpx.GPXTrackPoint]) -> dict:
    elevations = [p.elevation for p in points]
    times = [p.time for p in points]
    coords = [(p.latitude, p.longitude) for p in points]

    # 距離與坡度
    distances = [haversine(coords[i], coords[i+1], unit=Unit.METERS) for i in range(len(coords)-1)]
    elevation_diffs = [elevations[i+1] - elevations[i] for i in range(len(elevations)-1)]
    slopes = [np.degrees(np.arctan2(elev_diff, dist)) if dist > 0 else 0 for elev_diff, dist in zip(elevation_diffs, distances)]

    # 特徵彙總
    total_time = (times[-1] - times[0]).total_seconds() / 60  # 分鐘
    total_distance = sum(distances)
    elevation_range = (min(elevations), max(elevations))
    elevation_change = sum(e for e in elevation_diffs if e > 0), sum(e for e in elevation_diffs if e < 0)

    # 坡度分析
    slope_std = np.std(slopes)
    slope_var = np.var(slopes)
    max_slope_idx = np.argmax(np.abs(slopes))
    max_slope = slopes[max_slope_idx]

    # 坡度頻率分布
    slope_bins = [-90, -10, -5, 0, 5, 10, 90]
    slope_freq = pd.cut(slopes, bins=slope_bins).value_counts().sort_index()

    # 最大坡度高低時間差
    time_diff = (times[max_slope_idx+1] - times[max_slope_idx]).total_seconds()

    # 是否超過2438公尺
    over_2438 = any(e > 2438 for e in elevations)

    return {
        "total_time_min": total_time,
        "distance_m": total_distance,
        "elevation_gain_loss": elevation_change,
        "slope_std_dev": slope_std,
        "slope_variance": slope_var,
        "max_slope_deg": max_slope,
        "slope_freq_dist": slope_freq.to_dict(),
        "max_slope_time_diff_s": time_diff,
        "elevation_range_m": elevation_range,
        "over_2438m": over_2438,
        "max_slope_point": coords[max_slope_idx]
    }


In [2]:
def generate_html_map(points: List[gpxpy.gpx.GPXTrackPoint], output_file: str = 'map.html'):
    coords = [(p.latitude, p.longitude) for p in points]
    m = folium.Map(location=coords[0], zoom_start=15)
    folium.PolyLine(coords, color='blue', weight=3).add_to(m)
    folium.Marker(location=coords[0], popup='Start').add_to(m)
    folium.Marker(location=coords[-1], popup='End').add_to(m)
    m.save(output_file)


In [3]:
def generate_markdown_report(features: dict) -> str:
    md = f"""# GPX 路段特徵分析報告

## 1. 統計數據摘要

| 特徵名稱       | 數值                                      |
|----------------|-------------------------------------------|
| 總耗時間       | {features['total_time_min']:.2f} 分鐘       |
| 距離           | {features['distance_m']:.2f} 公尺           |
| 海拔變化       | 上升 {features['elevation_gain_loss'][0]:.2f} m / 下降 {abs(features['elevation_gain_loss'][1]):.2f} m |
| 坡度標準差     | {features['slope_std_dev']:.2f} 度         |
| 坡度變異數     | {features['slope_variance']:.2f}           |
| 最大坡度       | {features['max_slope_deg']:.2f} 度         |
| 海拔範圍       | {features['elevation_range_m'][0]:.2f} - {features['elevation_range_m'][1]:.2f} m |
| 超過2438m海拔? | {'是' if features['over_2438m'] else '否'} |
| 最大坡度點     | {features['max_slope_point']}              |
| 最大坡度時間差 | {features['max_slope_time_diff_s']:.2f} 秒 |

## 2. 坡度頻率分布

"""
    for k, v in features['slope_freq_dist'].items():
        md += f"- {k}: {v} 次\n"

    md += """
## 3. 定性觀察與詮釋

- 此段耗時與距離適中，坡度標準差與變異數顯示坡度變化明顯，對體力消耗有影響。
- 最大坡度接近 {0:.2f} 度，需注意該段行走安全。
- 海拔已超過2438m，進入高山區域，需注意高山症風險。

## 4. 指標定義與分類說明

- 「坡度效率」指坡度分佈是否均勻。若坡度標準差與變異數大，則屬「高負效率」（坡度變動劇烈）。
- 本段屬 **高負效率** 類型，建議規劃休息點並避開疲勞期登高。
""".format(features['max_slope_deg'])

    return md


In [4]:
gpx = load_gpx('標準桃山.gpx')
closest_idx = find_closest_point(gpx, (24.39700, 121.30770))
cut_points = slice_track(gpx, closest_idx, length=100)
features = compute_features(cut_points)
generate_html_map(cut_points, 'map.html')
report_md = generate_markdown_report(features)

# 儲存 Markdown
with open('report.md', 'w', encoding='utf-8') as f:
    f.write(report_md)


總路線圖


In [8]:
import xml.etree.ElementTree as ET
import folium

# GPX 檔案路徑
gpx_path = '標準桃山.gpx'

# 解析 GPX 檔案
tree = ET.parse(gpx_path)
root = tree.getroot()
ns = {'default': 'http://www.topografix.com/GPX/1/1'}
trkpts = root.findall('.//default:trkpt', ns)
coords = [(float(pt.attrib['lat']), float(pt.attrib['lon'])) for pt in trkpts]

# 參考點
ref_point = (24.39700, 121.30770)

# 建立 Folium 地圖
m = folium.Map(location=ref_point, zoom_start=13)

# 加入 GPX 路線
folium.PolyLine(coords, color='green').add_to(m)

# 加入起點與終點標記
folium.Marker(coords[0], popup='起點', icon=folium.Icon(color='blue')).add_to(m)
folium.Marker(coords[-1], popup='終點', icon=folium.Icon(color='blue')).add_to(m)

# 加入參考點標記
folium.Marker(ref_point, popup='參考點 24.39700, 121.30770', icon=folium.Icon(color='red')).add_to(m)

# 輸出 HTML 地圖
m.save('total_route_with_reference.html')
print("✅ 地圖已輸出：total_route_with_reference.html")


✅ 地圖已輸出：total_route_with_reference.html


切點顯示

In [9]:
import xml.etree.ElementTree as ET
import folium
import math

# GPX 檔案路徑
gpx_path = '標準桃山.gpx'

# 解析 GPX 檔案
tree = ET.parse(gpx_path)
root = tree.getroot()
ns = {'default': 'http://www.topografix.com/GPX/1/1'}
trkpts = root.findall('.//default:trkpt', ns)
coords = [(float(pt.attrib['lat']), float(pt.attrib['lon'])) for pt in trkpts]

# 參考點
ref_point = (24.39700, 121.30770)

# --- 新增功能：尋找最接近參考點的軌跡點 ---
min_dist = float('inf')
closest_index = -1

# 遍歷所有座標點，計算與參考點的距離
# 為了效率，這裡我們計算距離的平方，可以避免開根號，結果是一樣的
for i, point in enumerate(coords):
    dist_sq = (point[0] - ref_point[0])**2 + (point[1] - ref_point[1])**2
    if dist_sq < min_dist:
        min_dist = dist_sq
        closest_index = i

# 根據最接近的點，將路線切分為兩段
# 第一段：從起點到切分點
coords_part1 = coords[:closest_index + 1]
# 第二段：從切分點到終點
coords_part2 = coords[closest_index:]
# -----------------------------------------

# 建立 Folium 地圖
m = folium.Map(location=ref_point, zoom_start=13)

# --- 修改：分段繪製路線 ---
# 加入第一段路線 (綠色)
folium.PolyLine(coords_part1, color='green', weight=5, opacity=0.8, popup='第一段').add_to(m)
# 加入第二段路線 (橘色)
folium.PolyLine(coords_part2, color='orange', weight=5, opacity=0.8, popup='第二段').add_to(m)
# ---------------------------

# 加入起點與終點標記
folium.Marker(coords[0], popup='起點', icon=folium.Icon(color='blue')).add_to(m)
folium.Marker(coords[-1], popup='終點', icon=folium.Icon(color='blue')).add_to(m)

# 加入參考點標記 (紅色)
folium.Marker(ref_point, popup=f'參考點 {ref_point}', icon=folium.Icon(color='red')).add_to(m)

# --- 新增：加入切分點標記 (紫色) ---
if closest_index != -1:
    folium.Marker(
        coords[closest_index],
        popup=f'切分點 (最接近參考點)',
        icon=folium.Icon(color='purple')
    ).add_to(m)
# ------------------------------------

# 輸出 HTML 地圖
output_filename = 'split_route_map.html'
m.save(output_filename)
print(f"✅ 分段地圖已輸出：{output_filename}")

✅ 分段地圖已輸出：split_route_map.html


In [11]:
import xml.etree.ElementTree as ET
import folium
import numpy as np # 引入 numpy 進行向量運算

# GPX 檔案路徑
gpx_path = '「桃山」單攻.gpx'

# --- 幾何運算輔助函式 ---
def find_closest_point_on_segment(p1, p2, ref):
    """
    在由 p1 和 p2 定義的線段上，尋找最接近 ref 點的點。
    :param p1: 線段起點 (lat, lon)
    :param p2: 線段終點 (lat, lon)
    :param ref: 參考點 (lat, lon)
    :return: (最接近的點 (lat, lon), 到參考點的距離平方)
    """
    p1, p2, ref = np.array(p1), np.array(p2), np.array(ref)
    
    # 計算線段向量和長度的平方
    line_vec = p2 - p1
    line_len_sq = np.sum(line_vec**2)
    
    # 如果 p1 和 p2 是同一個點，直接回傳 p1 的距離
    if line_len_sq == 0:
        return p1, np.sum((ref - p1)**2)
        
    # 計算參考點在線段上的投影位置 (t)
    # t = dot((ref - p1), (p2 - p1)) / |p2 - p1|^2
    t = np.dot(ref - p1, line_vec) / line_len_sq
    
    # 根據 t 的值決定最近點的位置
    if t < 0:
        # 投影點在 p1 之外，所以最近點是 p1
        closest_point = p1
    elif t > 1:
        # 投影點在 p2 之外，所以最近點是 p2
        closest_point = p2
    else:
        # 投影點在線段之間，計算投影點座標
        closest_point = p1 + t * line_vec
        
    dist_sq = np.sum((ref - closest_point)**2)
    return tuple(closest_point), dist_sq

# --- 主程式開始 ---

# 解析 GPX 檔案
tree = ET.parse(gpx_path)
root = tree.getroot()
ns = {'default': 'http://www.topografix.com/GPX/1/1'}
trkpts = root.findall('.//default:trkpt', ns)
coords = [(float(pt.attrib['lat']), float(pt.attrib['lon'])) for pt in trkpts]

# 參考點
ref_point = (24.39700, 121.30770)

# --- 全新功能：尋找路徑上最接近參考點的「精確」位置 ---
min_dist_sq = float('inf')
true_closest_point = None
closest_segment_index = -1 # 記錄最近點所在的線段索引

# 遍歷所有「線段」(由兩個連續座標點組成)
for i in range(len(coords) - 1):
    p1 = coords[i]
    p2 = coords[i+1]
    
    # 尋找參考點到目前線段的最短距離點
    closest_pt_on_segment, dist_sq = find_closest_point_on_segment(p1, p2, ref_point)
    
    if dist_sq < min_dist_sq:
        min_dist_sq = dist_sq
        true_closest_point = closest_pt_on_segment
        closest_segment_index = i

# --- 根據最接近的「精確點」將路線切分為兩段 ---
# 第一段：從起點到最接近點
# 包含開頭到最近線段的起點，再加上精確計算出的最接近點
coords_part1 = coords[:closest_segment_index + 1] + [true_closest_point]

# 第二段：從最接近點到終點
# 包含精確計算出的最接近點，再加上最近線段的終點到結尾
coords_part2 = [true_closest_point] + coords[closest_segment_index + 1:]

# --- 建立 Folium 地圖 ---
m = folium.Map(location=ref_point, zoom_start=13)

# --- 分段繪製路線 ---
folium.PolyLine(coords_part1, color='green', weight=5, opacity=0.8, popup='第一段').add_to(m)
folium.PolyLine(coords_part2, color='orange', weight=5, opacity=0.8, popup='第二段').add_to(m)

# --- 繪製標記 ---
# 起點與終點
folium.Marker(coords[0], popup='起點', icon=folium.Icon(color='blue')).add_to(m)
folium.Marker(coords[-1], popup='終點', icon=folium.Icon(color='blue')).add_to(m)

# 參考點 (紅色)
folium.Marker(ref_point, popup=f'參考點 {ref_point}', icon=folium.Icon(color='red')).add_to(m)

# 精確切分點 (紫色)
if true_closest_point:
    folium.Marker(
        true_closest_point,
        popup=f'精確切分點 (路徑上最接近參考點的位置)',
        icon=folium.Icon(color='purple')
    ).add_to(m)
    
    # 新增：畫一條虛線連接參考點與最近點，視覺化最短距離
    folium.PolyLine(
        [ref_point, true_closest_point],
        color='red',
        weight=2,
        opacity=0.8,
        dash_array='5, 5' # 虛線樣式
    ).add_to(m)

# 輸出 HTML 地圖
output_filename = '桃山單攻_split_route_map_precise.html'
m.save(output_filename)
print(f"✅ 精確分段地圖已輸出：{output_filename}")

✅ 精確分段地圖已輸出：桃山單攻_split_route_map_precise.html


In [ ]:
桃山單攻多點位

In [19]:
import folium
import gpxpy
import gpxpy.gpx
import numpy as np
import io

# --- 1. 資料輸入 ---

# 將您在提示中提供的點位資料讀取進來
points_data_str = """
步道名稱	路標指示	緯度	經度	海拔（約）	是否為切點
桃山瀑布步道	0 k 起點	24.39700	121.30770	1400 m	Y
桃山步道	0K_瀑布1K交會	24.40522	121.30758	2100 m	Y
桃山步道	1 K	24.41011	121.31125	2300 m	N
桃山步道	1.5 K	24.41304	121.30947	2500 m	N
桃山步道	2 K	24.41630	121.30720	2700 m	Y
桃山步道	3.5 K	24.42640	121.30377	3000 m	N
桃山步道	4 K	24.42911	121.30409	3050 m	N
桃山步道	4.5 K	24.43251	121.30463	3323 m	Y
"""

# GPX 檔案路徑 (請確保此檔案與您的 Python 腳本在同一個資料夾)
gpx_filename = '標準桃山.gpx'

# --- 2. 輔助函式與資料處理 ---

def find_closest_point_on_segment(p1, p2, ref):
    """
    在由 p1 和 p2 定義的線段上，尋找最接近 ref 點的點。
    返回: (最接近的點 (lat, lon), 到參考點的距離平方, 在線段上的投影位置 t)
    """
    p1, p2, ref = np.array(p1), np.array(p2), np.array(ref)
    line_vec = p2 - p1
    line_len_sq = np.sum(line_vec**2)

    if line_len_sq == 0:
        return p1, np.sum((ref - p1)**2), 0.0

    t = np.dot(ref - p1, line_vec) / line_len_sq

    if t < 0:
        closest_point = p1
    elif t > 1:
        closest_point = p2
    else:
        closest_point = p1 + t * line_vec

    dist_sq = np.sum((ref - closest_point)**2)
    return tuple(closest_point), dist_sq, t

def parse_points_data(data_str):
    """解析字串格式的點位資料"""
    points = []
    lines = data_str.strip().split('\n')
    header = lines[:1][0].split('\t')
    for line in lines[1:]:
        values = line.split('\t')
        point_info = {
            'name': values[0],
            'label': values[1],
            'lat': float(values[2].replace('°', '')),
            'lon': float(values[3].replace('°', '')),
            'is_cut_point': values[5].strip().upper() == 'Y'
        }
        points.append(point_info)
    return points

# --- 3. 主程式邏輯 ---

# 讀取 GPX 檔案
try:
    with open(gpx_filename, 'r', encoding='utf-8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
except FileNotFoundError:
    print(f"錯誤：GPX 檔案 '{gpx_filename}' 不存在。請確認檔案名稱和路徑。")
    exit()

# 提取軌跡座標
track_coords = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            track_coords.append((point.latitude, point.longitude))

# 解析點位資料
all_points = parse_points_data(points_data_str)
cut_points_info = [p for p in all_points if p['is_cut_point']]

# 尋找每個切點在軌跡上的精確投影位置
projected_cut_points = []
for cp_info in cut_points_info:
    ref_point = (cp_info['lat'], cp_info['lon'])
    min_dist_sq = float('inf')
    best_projection = None

    for i in range(len(track_coords) - 1):
        p1, p2 = track_coords[i], track_coords[i+1]
        proj_point, dist_sq, t = find_closest_point_on_segment(p1, p2, ref_point)

        if dist_sq < min_dist_sq:
            min_dist_sq = dist_sq
            best_projection = {
                'info': cp_info,
                'projected_point': proj_point,
                'segment_index': i,
                'projection_t': t
            }
    if best_projection:
        projected_cut_points.append(best_projection)

# 根據在軌跡上的前後順序對切點進行排序
projected_cut_points.sort(key=lambda x: (x['segment_index'], x['projection_t']))

# 建立分割點列表，包含起點、所有排序後的投影切點、終點
split_nodes = [track_coords[0]]
split_nodes.extend([p['projected_point'] for p in projected_cut_points])
split_nodes.append(track_coords[-1])
# 去除可能因投影產生的重複點
split_nodes = sorted(list(set(split_nodes)), key=lambda x: split_nodes.index(x))

# --- 4. 建立 Folium 地圖 ---

# 以軌跡的平均位置為中心建立地圖
map_center = np.mean(track_coords, axis=0)
m = folium.Map(location=map_center, zoom_start=14)

# 定義路段顏色
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
          'lightblue', 'darkgreen', 'cadetblue', 'darkpurple']

# 繪製分割後的路段
# current_track_index = 0
# for i in range(len(split_nodes) - 1):
#     start_node = split_nodes[i]
#     end_node = split_nodes[i+1]

#     segment_coords = [start_node]

#     while current_track_index < len(track_coords):
#         if track_coords[current_track_index] == start_node:
#             break
#         current_track_index += 1

#     for j in range(current_track_index + 1, len(track_coords)):
#         point = track_coords[j]
#         segment_coords.append(point)
#         if point == end_node:
#             break

#     if segment_coords[-1] != end_node:
#         if end_node not in segment_coords:
#             segment_coords.append(end_node)

#     folium.PolyLine(
#         segment_coords,
#         color=colors[i % len(colors)],
#         weight=6,
#         opacity=0.8,
#         popup=f'路段 {i+1}'
#     ).add_to(m)

# 繪製所有點位的標記
for point in all_points:
    popup_html = f"<b>{point['name']}</b><br>{point['label']}"
    if point['is_cut_point']:
        # 對於切點，使用紫色圖示
        folium.Marker(
            location=(point['lat'], point['lon']),
            popup=popup_html,
            icon=folium.Icon(color='purple', icon='cut', prefix='fa')
        ).add_to(m)
    else:
        # 非切點使用藍色圖示
        folium.Marker(
            location=(point['lat'], point['lon']),
            popup=popup_html,
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

# 只畫整體軌跡路線，無分段色彩
folium.PolyLine(
    track_coords,
    color='blue',
    weight=4,
    opacity=0.7,
    popup='完整軌跡'
).add_to(m)
# --- 移除連接線 ---
# 原先的連接線繪製程式碼如下，現已註解：
# for pcp in projected_cut_points:
#     folium.PolyLine(
#         [[pcp['info']['lat'], pcp['info']['lon']], pcp['projected_point']],
#         color='red',
#         weight=2,
#         opacity=0.9,
#         dash_array='5, 5'
#     ).add_to(m)

# --- 5. 儲存地圖 ---
output_filename = '桃園桃山多切點路徑圖_無連接線.html'
m.save(output_filename)

print(f"✅ 地圖已成功建立並儲存為：{output_filename}")


✅ 地圖已成功建立並儲存為：桃園桃山多切點路徑圖_無連接線.html
